In [1]:
import os
import sys
module_path_dont_know = os.path.abspath(os.path.join('../COBRAS_dont_know'))

if module_path_dont_know not in sys.path:
    sys.path.append(module_path_dont_know)
    print("module path of dont_know added")

from cobras.constraints.constraint import Constraint
from cobras.constraints.constraint_type import ConstraintType
from scipy.spatial import distance

from cobras.cobras import COBRAS
from cobras.cobras_logger import COBRASLogger
from heuristics.splitlevel_estimation_strategy import StandardSplitLevelEstimationStrategyAlwayskmeans
from cobras.querier.labelquerier import LabelQuerier
from util.datasets import Dataset
from heuristics.select_super_instance_heuristics import SelectMostInstancesHeuristic
from clustering_algorithms.kmeans_fixed_representative import KmeansFixedRepresentative
from cobras.super_instances.superinstance_select_representative import SuperInstance_select_representative_Builder

module path of dont_know added


In [2]:
def get_dissimilarity(c1: Constraint, c2: Constraint):
    length1 = distance.euclidean(c1.i1, c1.i2)
    length2 = distance.euclidean(c2.i1, c2.i2)
    denominator = (length1 + length2)/2
    distance1 = distance.euclidean(c1.i1, c2.i1)
    distance2 = distance.euclidean(c1.i2, c2.i2)
    distance3 = distance.euclidean(c1.i1, c2.i2)
    distance4 = distance.euclidean(c1.i2, c2.i1)
    numerator = min(max(distance1, distance2), max(distance3, distance4))
    return (numerator / denominator)

In [3]:
c1 = Constraint((1,3),(4,3), ConstraintType.DK)
c2 = Constraint((1,2),(4,3), ConstraintType.DK)
c3 = Constraint((30,10),(4,3), ConstraintType.DK)

sim1 = get_dissimilarity(c1, c2)
sim2 = get_dissimilarity(c1, c3)
print(sim1)
print(sim2)

0.32455532033675866
1.7995042678574928
